In [15]:
import pandas as pd

In [19]:
ubt = pd.read_csv('ubt.csv')

In [3]:
params = pd.read_csv('data.csv')
params.head(2)

,name,build_power,speed,e_storage,m_storage,e_production,m_production,m_cost,e_cost,bp300bt,bp80bt,max_count
0,initial,NaN,NaN,500.0,500.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,com,300.0,NaN,500.0,500.0,25.0,2.0,2700.0,26000.0,NaN,NaN,NaN


In [4]:
data = {
        'entity': ['com', 'mex', 'solar', 'wind','con', 'mc', 'nano', 'solar_adv'],
        'count': [1, 3, 2, 3, 1, 0, 0, 0]
       }

df = pd.DataFrame(data)
df.to_csv('initial_t1.csv')
df.head(2)

,entity,count
0,com,1
1,mex,3


In [27]:
# Merge the two dataframes based on the 'entity' and 'name' columns
merged_df = pd.merge(df, params[[
    'name', 
    'build_power', 
    'e_production',
    'm_production',
    'e_cost',
    'm_cost'
]], left_on='entity', right_on='name', how='left')
merged_df['bp_amount']=merged_df.build_power*merged_df['count']
merged_df

,entity,count,name,build_power,e_production,m_production,e_cost,m_cost,bp_amount
0,com,1,com,300.0,25.0,2.0,26000.0,2700.0,300.0
1,mex,3,mex,NaN,-3.0,1.8,500.0,50.0,NaN
2,solar,2,solar,NaN,20.0,NaN,0.0,150.0,NaN
3,wind,3,wind,NaN,8.0,NaN,175.0,45.0,NaN
4,con,1,con,80.0,NaN,NaN,1750.0,125.0,80.0
5,mc,0,mc,NaN,-70.0,1.0,1250.0,1.0,NaN
6,nano,0,nano,200.0,NaN,NaN,3200.0,210.0,0.0
7,solar_adv,0,solar_adv,NaN,75.0,NaN,4000.0,370.0,NaN


In [28]:
merged_df.bp_amount.sum()

380.0

In [29]:
steps = {
        'e_amount': [272],
        'm_amount': [248],
        'e_production': [99],
        'm_production': [8],
        'order': [''],
        'time':[0]
       }
steps = pd.DataFrame(steps)
steps

,e_amount,m_amount,e_production,m_production,order,time
0,272,248,99,8,,0


In [34]:
candidates = ['solar','wind','solar_adv','mc']
step = 0
# Choose the candidates that are available
for candidate in candidates:
    row = merged_df[merged_df.entity==candidate].iloc()[0]
    step_row = steps.iloc()[step]
    e_cost = row.e_cost
    m_cost = row.m_cost
    e_amount = step_row.e_amount
    m_amount = step_row.m_amount
    if e_cost > e_amount or m_cost > m_amount:
        # print('not enought resources:', candidate, e_cost, m_cost)
        continue
    
    overall_e_production = (merged_df.e_production * merged_df['count']).sum()
    entity_e_production = row.e_production
    build_power_sum = merged_df.bp_amount.sum()
    # Calculate the time of building
    bt = ubt[ubt.unit==candidate].iloc()[0].unit_build_time_calculated/build_power_sum
    # Calculate the value of E that will be produced during the building time
    e_prod_amount = merged_df.e_production.sum() * bt
    e_final_amount = e_amount + e_prod_amount
    # Calculate the value of M that will be produced during the building time
    m_prod_amount = merged_df.m_production.sum() * bt
    m_final_amount = m_amount + m_prod_amount

    # e_final_production = entity_e_production + row.e_production
    
    print(f"""
# {candidate} 
e_cost: {e_cost}
m_cost: {m_cost}
entity_e_production: {entity_e_production}
build_time: {bt}
current_e_prod: {merged_df.e_production.sum()}
overall_e_production: {overall_e_production}
e_final_amount: {e_final_amount}
m_final_amount: {m_final_amount}""")


# solar 
e_cost: 0.0
m_cost: 150.0
entity_e_production: 20.0
build_time: 7.894736842105263
current_e_prod: 55.0
overall_e_production: 80.0
e_final_amount: 706.2105263157895
m_final_amount: 285.89473684210526

# wind 
e_cost: 175.0
m_cost: 45.0
entity_e_production: 8.0
build_time: 4.2105263157894735
current_e_prod: 55.0
overall_e_production: 80.0
e_final_amount: 503.57894736842104
m_final_amount: 268.2105263157895


['solar', 'wind']

In [9]:
# new_row = {
#     'e_amount': 300,
#     'm_amount': 275,
#     'e_production': 120,
#     'm_production': 10,
#     'order': 'sol',
#     'time': 1
# }
# steps.loc[len(steps)] = new_row